In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import UnexpectedAlertPresentException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.alert import Alert
import re
from datetime import datetime
import time


In [2]:
def selenium_init():
    driver = webdriver.Chrome('chromedriver.exe')
    driver.implicitly_wait(5)
    return driver

In [3]:
def login(driver, id, pwd, musical_id):
    
    interpark_login_url = 'https://ticket.interpark.com/Gate/TPLogin.asp?CPage=B&MN=Y&tid1=main_gnb&tid2=right_top&tid3=login&tid4=login&GPage=https%3A%2F%2Ftickets.interpark.com%2Fgoods%2F' + musical_id
    driver.get(interpark_login_url)

    driver.switch_to.frame(driver.find_element(By.XPATH, "//div[@class='leftLoginBox']/iframe[@title='login']"))
    driver.find_element_by_name('userId').send_keys(id)
    driver.find_element_by_name('userPwd').send_keys(pwd)
    driver.find_element_by_id('btn_login').click()

In [4]:
# 태그가 없으면 에러발생
def check_exists_by_element(driver, by, name):
    elements = driver.find_elements(by, name)
    print(elements)
    if not elements:
        return False
    else:
        return True


In [5]:
def close_tickting_info_popup(driver):
    # 예매안내가 팝업이 뜨면 닫기. ( ticketingInfo_check : True, False )
    ticketinginfo_check = check_exists_by_element(driver, By.XPATH, "//div[@class='popupWrap']/div[@class='popupFooter']/button[@class='popupCloseBtn is-bottomBtn']")
    if ticketinginfo_check:
        popup_close_btn = driver.find_element(By.XPATH, "//button[@class='popupCloseBtn is-bottomBtn']")
        if popup_close_btn.text != "":
            popup_close_btn.click()

In [6]:
def close_ticketing_check_popup(driver):
    # 예매안내가 팝업이 뜨면 닫기. ( ticketingInfo_check : True, False )
    driver.switch_to.default_content()
    ticketingInfo_check = check_exists_by_element(driver, By.XPATH, "//div[@class='layerWrap']/div[@class='titleArea']/a[@class='closeBtn']")
    if ticketingInfo_check:
        driver.find_element(By.XPATH, "//div[@class='layerWrap']/div[@class='titleArea']/a[@class='closeBtn']").click()

In [7]:
def clickWantDate2(driver, wantYear, wantMonth, wantDay, wantTime):
    year_month = driver.find_element(By.XPATH, "//ul/li[@data-view='month current']").text.split('.')
    year = year_month[0].strip()
    month = year_month[1].strip()
    
    year_diff = wantYear - int(year)
    month_diff = wantMonth - int(month)
    
    total_month_diff = year_diff * 12 + month_diff
    
    # 원하는 월이 될 때까지 이동
    for i in range(abs(total_month_diff)):
        if total_month_diff > 0:
            next_month = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//ul/li[@data-view='month next']")))
            driver.execute_script("arguments[0].click();", next_month)
#           # next_month = driver.find_element(By.XPATH, "//ul/li[@data-view='month next']")
            print(next_month.text)
            
#             next_month.click()
        else:
            prev_month = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//ul/li[@data-view='month prev']")))
            driver.execute_script("arguments[0].click();", prev_month)
#           # prev_month = driver.find_element(By.XPATH, "//ul/li[@data-view='month prev']")
#             prev_month.click()
    
    # 원하는 일자 클릭
#     time.sleep(0.1)
#     day = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//div[@class='datepicker-panel']/ul[@data-view='days']/li[text()='" + str(wantDay) + "']")))
#     print(day.text)
    days = driver.find_elements(By.XPATH, "//div[@class='datepicker-panel']/ul[@data-view='days']/li")
    for day in days:
            if int(day.text) == wantDay:	# wantDate : 예매 원하는 일
                day.click()
                break
#     driver.execute_script("arguments[0].click();", day)
#     day.click()

            
    # 원하는 시간 클릭
    show_times = driver.find_elements(By.XPATH, "//ul[@class='timeTableList']/li")
    for show_time in show_times:
        if show_time.text.split(' ')[1] == wantTime:
            show_time.click()
            break
    
    

In [8]:
def click_book_btn(driver):
    print(driver.find_element(By.XPATH, "//li[@data-view='month current']").text)
    
#     (driver.find_element(By.XPATH, "//li[@data-view='month next']").text)
    
    # 예매하기 버튼 클릭
    driver.find_element(By.XPATH, "//div[@class='sideBtnWrap']/a[@class='sideBtn is-primary']").click()


In [9]:
def switching_focus(driver):
    # 예매하기 눌러서 새창이 뜨면 포커스를 새창으로 변경
    driver.switch_to.window(driver.window_handles[1])
    driver.get_window_position(driver.window_handles[1])

In [10]:
def waitSafeBooking(driver):
    driver.switch_to.default_content()
    iframe_element = WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.XPATH, "//div[@id='divBookSeat']/iframe[@id='ifrmSeat']")))
    driver.switch_to.frame(iframe_element)
    
    recaptcha_input = driver.find_elements(By.XPATH, "//div[@class='validationTxt']")
    recaptcha_input[0].click()
    
    video_element = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, "//div[@id='divRecaptcha' and @style='display: none;']")))
    print("안심예매 RECAPTCHA 눌렀다!")
    

In [11]:
def clickWantSeat(driver):
    driver.switch_to.default_content()
    driver.switch_to.frame(driver.find_element(By.XPATH, "//div[@id='divBookSeat']/iframe[@id='ifrmSeat']"))
    driver.switch_to.frame(driver.find_element(By.XPATH, "//div[@class='seatL']/iframe[@id='ifrmSeatDetail']"))
    stySeats = driver.find_elements(By.XPATH, "//img[@class='stySeat']")
    for seat in stySeats:
        seatInfo = seat.get_attribute("onclick").split(',')

        floor = int(re.findall('\d+', seatInfo[2])[0])
        column = int(re.findall('\d+', seatInfo[3])[0])
        row = int(re.findall('\d+', seatInfo[4])[0])
        print("여기 뜨는게 없다고?", floor, column, row)

        if floor == 1 and column <= 25 and 9 <= row <= 26:
            print(seat.text)
            seat.click()
            break
    
    driver.switch_to.default_content()
    driver.switch_to.frame(driver.find_element(By.XPATH, "//div[@id='divBookSeat']/iframe[@id='ifrmSeat']"))
    nextBtn = driver.find_element(By.XPATH, "//div[@class='btnWrap']/a")
    print(nextBtn.text)
    nextBtn.click()
    
    try:
        WebDriverWait(driver, 1).until(EC.alert_is_present(),
                                       'Timed out waiting for PA creation ' +
                                       'confirmation popup to appear.')

        alert = driver.switch_to.alert
#         if alert.is_displayed():
        if alert.text.strip() == '좌석을 선택하세요.':
            print("좌선 크크아아 선택")
        alert.accept()
        print("alert accepted")
        clickWantSeat(driver)
     
    except TimeoutException:
        print("no alert")
        

In [12]:
def clickTicketPrice(driver):
    driver.switch_to.default_content()
    driver.switch_to.frame(driver.find_element(By.XPATH, "//div[@class='contL']/iframe[@id='ifrmBookStep']"))
    ticketNums = driver.find_elements(By.XPATH, "//tr[@id='PriceRow001']/td[@class='taL']/select/option")
    for ticketNum in ticketNums:
        num = int(re.findall('\d+', ticketNum.text)[0])
        if num == 1:
            ticketNum.click()
            break

    driver.switch_to.default_content()
    driver.find_element(By.XPATH, "//p[@id='SmallNextBtn']").click()

In [13]:
def confirmOderInfo(driver, birthDay, phoneNumber):
    driver.switch_to.default_content()
    driver.switch_to.frame(driver.find_element(By.XPATH, "//div[@class='contL']/iframe[@id='ifrmBookStep']"))
    driver.find_element_by_id('YYMMDD').clear()
    driver.find_element_by_id('YYMMDD').send_keys(birthDay)
    driver.find_element_by_id('HpNo1').clear()
    driver.find_element_by_id('HpNo1').send_keys(phoneNumber[0])
    driver.find_element_by_id('HpNo2').clear()
    driver.find_element_by_id('HpNo2').send_keys(phoneNumber[1])
    driver.find_element_by_id('HpNo3').clear()
    driver.find_element_by_id('HpNo3').send_keys(phoneNumber[2])

    driver.switch_to.default_content()
    driver.find_element(By.XPATH, "//p[@id='SmallNextBtn']").click()

In [14]:
def clickPaymentMethod(driver, wantBank):
    driver.switch_to.default_content()
    driver.switch_to.frame(driver.find_element(By.XPATH, "//div[@class='contL']/iframe[@id='ifrmBookStep']"))
    
    # 무통장입금 클릭(매칭 id로 검색)
    # 유연하게 진행하기 위해선 결제방식에 따른 id값을 미리 가지고 있을 필요있음
    driver.find_element(By.XPATH, "//tr[@id='Payment_22004']/td/input").click()

    # 입금 은행 - 농협 클릭
    banks = driver.find_elements(By.XPATH, "//select[@id='BankCode']/option")
    for bank in banks:
        if bank.text == wantBank:
            bank.click()
            break

    driver.switch_to.default_content()
    driver.find_element(By.XPATH, "//p[@id='SmallNextBtn']").click()

In [15]:
def agreePayment(driver):
    driver.switch_to.default_content()
    driver.switch_to.frame(driver.find_element(By.XPATH, "//div[@class='contL']/iframe[@id='ifrmBookStep']"))
    driver.find_element_by_id('checkAll').click()

    driver.switch_to.default_content()
    driver.find_element(By.XPATH, "//p[@id='LargeNextBtn']").click()

In [16]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QLineEdit, QDateTimeEdit, QPushButton, QVBoxLayout
from PyQt5.QtCore import Qt, QDate, QDateTime
from PyQt5.QtGui import QIcon

class MyApp(QWidget):

    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setWindowTitle('SwimmingLee Interpark Macro')
        self.setWindowIcon(QIcon('swim_con.png'))
        
        self.ticketing_info = QLabel('시범용입니다. 하하 ', self)
        self.ticketing_info.setAlignment(Qt.AlignCenter)
        
        self.ticketing_info_font = self.ticketing_info.font()
        self.ticketing_info_font.setPointSize(20)
        self.ticketing_info.setFont(self.ticketing_info_font)
        
        
        self.date_time_label = QLabel('Date: ')
        self.date_time_edit = QDateTimeEdit(self, calendarPopup=True)
        self.date_time_edit.setDateTime(QDateTime.currentDateTime())
        self.date_time_edit.setMinimumDateTime(QDateTime(2021, 1, 1, 00, 00, 00))
        self.date_time_edit.setMaximumDateTime(QDateTime(2030, 12, 31, 00, 00, 00))
        self.date_time_edit.setDisplayFormat('yyyy.MM.dd hh:mm')
        
        self.id_label = QLabel('ID: ')
        self.id_edit = QLineEdit(self)
        self.id_edit.setPlaceholderText("Set your Interpark ID")
        
        self.pw_label = QLabel('PW: ')
        self.pw_edit = QLineEdit(self)
        self.pw_edit.setEchoMode(QLineEdit.Password)
        self.pw_edit.setPlaceholderText("Set your Interpark password")
        
        self.show_label = QLabel('Show Code: ')
        self.show_edit = QLineEdit(self)
        self.show_edit.setPlaceholderText("Set your Interpark show code")
#         show_edit.setValidator(QIntValidator())
        
        
        self.login_btn = QPushButton('Login', self)
        self.login_btn.setCheckable(True)
        self.login_btn.clicked.connect(self.login)
        
        self.ticketing_btn = QPushButton('Start', self)
        self.ticketing_btn.setCheckable(True)
        self.ticketing_btn.clicked.connect(self.ticketingStart)
        
        
        vbox = QVBoxLayout()
        
        vbox.addWidget(self.ticketing_info)
        
        vbox.addWidget(self.id_label)
        vbox.addWidget(self.id_edit)
        
        vbox.addWidget(self.pw_label)
        vbox.addWidget(self.pw_edit)
        
        vbox.addWidget(self.show_label)
        vbox.addWidget(self.show_edit)
        
        vbox.addWidget(self.date_time_label)
        vbox.addWidget(self.date_time_edit)
        
        vbox.addStretch()
        
        vbox.addWidget(self.login_btn)
        vbox.addWidget(self.ticketing_btn)
        
        self.setLayout(vbox)
        
        self.setGeometry(300, 300, 300, 200)
        self.move(300, 300)
        self.resize(400, 200)
        self.show()

    def login(self):
        id = self.id_edit.text()
        pw = self.pw_edit.text()
        show_id = self.show_edit.text()
        
        want_date = self.date_time_edit.date()
        want_year = want_date.year()
        want_month = want_date.month()
        want_day = want_date.day()  
        want_time = self.date_time_edit.time().toString('hh:mm')
        
        self.driver = selenium_init()
        login(self.driver, id, pw, show_id)
        close_tickting_info_popup(self.driver)
        clickWantDate2(self.driver, want_year, want_month, want_day, want_time)
        
        
    def ticketingStart(self):
        
        self.driver.switch_to.window(self.driver.window_handles[0])
        self.driver.get_window_position(self.driver.window_handles[0])
    
    
        click_book_btn(self.driver)
        switching_focus(self.driver)
        close_ticketing_check_popup(self.driver)
        waitSafeBooking(self.driver)

        clickWantSeat(self.driver)
        clickTicketPrice(self.driver)
        confirmOderInfo(self.driver, '930111', ['010', '7294', '6010'])
        clickPaymentMethod(self.driver, '농협(중앙)')

In [17]:
app = QApplication(sys.argv)
ex = MyApp()
sys.exit(app.exec_())

[]


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=91.0.4472.164)


[<selenium.webdriver.remote.webelement.WebElement (session="521f26292d5354d4930d0a0f36f51bc2", element="0716805a-7585-420a-8421-f70acf2894cc")>]


C:\Users\swimm\anaconda3\envs\musical\lib\site-packages\selenium\webdriver\remote\webdriver.py:1149: UserWarning: Only 'current' window is supported for W3C compatibile browsers.
  warnings.warn("Only 'current' window is supported for W3C compatibile browsers.")


2021. 07
[<selenium.webdriver.remote.webelement.WebElement (session="521f26292d5354d4930d0a0f36f51bc2", element="8db95742-4218-4e7f-90ab-db1efe690bb4")>]


IndexError: list index out of range

SystemExit: 0

C:\Users\swimm\anaconda3\envs\musical\lib\site-packages\IPython\core\interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
안심예매인지 아닌지에 따라서 생기는 팝업창이 완전 다른거 같음..!? 
암튼 유연하게 대처할 수 있는 방법이 필요함

안심예매인지 아닌지 자동으로 판별하는 로직이 필요할듯함